# Sentiment Analysis Multi-Language

## 1. Initial Setup

In [ ]:
# Setup environment Windows
# python -m venv .venv_sentiment
# .venv_sentiment\Scripts\Activate.ps1  

# Linux
# python3 -m venv .venv_sentiment
# source awesome_venv/bin/activate

# pip install -r requirements.txt
# python3 -m pip install tensorflow[and-cuda]
# pip install 'transformers[tf-cpu]'
# pip install 'transformers[flax]'


In [1]:
import argparse
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import os

/home/ubuntu/experiments/thesis_GenAI/sentiment/.venv_sentiment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
# Import comments
path = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets"
file_name = "comments_cleaned.csv"
file_path = os.path.join(path, file_name)
df_comments = pd.read_csv(file_path)
df_comments.head()

,stream_name,created_date,post_id,comment_id,comment_text,like_count,report_count,username,author_user_id,author_position,author_status
0,Water Break,2024-03-05 15:15:55,7612625,6866587,Congratulations and a great addition to the te...,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
1,Water Break,2024-03-05 15:10:47,7612663,6866567,"Hello team, remember to have safety discussion...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
2,Water Break,2024-03-05 15:08:29,7613861,6866561,"@Larry_Broderick, both our moderation team and...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
3,Safety & Operations,2024-03-05 15:04:42,7613237,6866545,"@William_Pulliam, we appreciate you being prou...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
4,Safety & Operations,2024-03-05 15:02:47,7613255,6866539,"We appreciate you supporting @Dennis_Cumbie, @...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active


In [11]:
parser = argparse.ArgumentParser("Sentiment Analysis")
parser.add_argument('comments_cleaned', help='dataframe with the text to analysis on pandas frame format')
parser.add_argument('comment_text', help='column in the file that contains the text')
args = parser.parse_args()

df_text = pd.read_csv(args.comments_cleaned)#, lineterminator='\n')
# model = 'finiteautomata/bertweet-base-sentiment-analysis'
model = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
sentiment = pipeline(task='sentiment-analysis', model=model)
counter = 0

# Define the total number of iterations
total_iterations = len(df_text)
# Create a progress bar
progress_bar = tqdm(total=total_iterations, position=0, leave=True)

for index, row in df_text.iterrows():
    try:
        df_text.loc[index, 'sentiment'] = str(sentiment(str(row[args.comment_text])[:279])[0]['label'])
        df_text.loc[index, 'sentiment_score'] = float(sentiment(str(row[args.comment_text])[:279])[0]['score'])
    except Exception as e:
        print(f'{str(row[args.comment_text])}', f'Error -> {e}')
    counter += 1
    progress_bar.update(1)  # Update the progress bar
    progress_bar.set_description(f'Processed: {counter}/{total_iterations}')

df_text.to_csv(f'sentiment_analysis_{args.file}', index=False)

usage: Sentiment Analysis [-h] comments_cleaned comment_text
Sentiment Analysis: error: the following arguments are required: comments_cleaned, comment_text


SystemExit: 2